In [1]:
data_dir = "data/"
#pr_fname = "test.csv"
pr_fname = "pressrelease_all.csv"
car_fname = "car.new"

In [ ]:
import unicodedata

pr = {}
with open(data_dir + pr_fname, 'r') as f:
    for line in f:
        articleid, date, bodysub = line.strip().split('\t')
        pr[articleid] = unicodedata.normalize('NFKC', bodysub.replace('　', ''))
        
#pr
# Doc2Vecの学習用に全て与えたい

In [3]:
from collections import defaultdict

car_all = defaultdict(list)
with open(car_fname, 'r') as f:
    for line in f:
        articleid, prtype, code, val = line.strip().split('\t')
        car_all[articleid].append(float(val))
        
car = {}
for articleid, val_list in car_all.items():
    car[articleid] = sum(val_list) / len(val_list)
    
#car

In [4]:
import subprocess
import os

In [5]:
# JUMAN++は文章の最後の方になると一文字づつ分解するようになってしまった(変な文字が多いせい？)のでMeCabを使用。

def run_mecab(data):
    if isinstance(data, tuple):
        articleid, s = data
    else:
        s = data
    
    if isinstance(data, tuple):
        fname = "tmp." + articleid
    else:
        fname = "tmp." + str(os.getpid())
    with open(fname, 'w') as f:
        f.write(s)
    subprocess.call("mecab -O wakati " + fname + " > out." + fname, shell = True)   # MeCabがバッファが足りないと言うので標準エラー出力を排除
    ret = subprocess.getoutput("head -1 out." + fname + " | iconv -c -t UTF-8").strip()   #  iconvでUTF-8変換できないところを排除、上に関連？
    subprocess.call("rm " + fname + " out." + fname, shell = True)
        
    #ret_split = [x.split('\t')[0] for x in ret][:-1]
    if isinstance(data, tuple):
        return (articleid, ret)
    else:
        return ret

In [6]:
print(run_mecab((list(pr.keys())[0], list(pr.values())[0])))

('NIKPRLRSP038042_07012003', '発表 日 : 2003 年 1 月 7 日 マイクロ ドライブ を 4 ギガバイト に 拡張 株式会社 日立 グローバルストレージテクノロジーズ ( 取締役 社長 : 宮崎 哲男 )(* 1 ) は 、 世界 最小 の ハードディスク 装置 「 マイクロ ドライブ 」 の 記憶 容量 を 4 ギガバイト に 拡張 し 、 2003 年 秋 に 発売 し ます 。 (* 1 ) 株式会社 日立 グローバルストレージテクノロジーズ は H i t a c h i GlobalStorageTechnologies ( CEO : 成瀬 淳 、 本社 機能 : 米国 カリフォルニア 州 ) の 日本 法人 として 、 2003 年 1 月 1 日 に 設立 さ れ まし た 。 同社 は ハードディスク 装置 の 開発 ・ 製造 ならびに 国内 における 販売 を 担当 し ます 。 マイクロ ドライブ は 、 標準 的 な メモリー の 規格 で ある コンパクト ・ フラッシュ TypeII の 物理 的 条件 を 満たし て おり 、 外形 寸法 は 幅 42 . 8 mm× 奥行き 36 . 4 mm× 厚 さ 5 . 0 mm という 小型 サイズ です 。 米国 IBM 社 が 2000 年 6 月 に 記憶 容量 1 ギガバイト の 製品 を 発売 し て 以来 、 大 容量 化 を 目指し て 開発 を 進め て き た 結果 、 小型 化 技術 を 活用 する こと で 、 読み書き ヘッド が 従来 製品 の 半分 の サイズ と なる 「 FEMTO (* 2 ) ヘッド 」 を 新た に 開発 する とともに 、 ヘッド が 円 板 上 を 浮上 する 高 さ ( フライト・ハイト ) も 約 40 % 下げる こと に 成功 し まし た 。 また 、 「 ピクシー・ダスト ( PixieDust )」 技術 を 利用 し て 線 記録 密度 を 大幅 に 向上 さ せ て い ます 。 更に 、 トラック 密度 の 向上 と 併せ て 面 記録 密度 も 100 億 ビット / 平方センチメートル に 向上 さ せ まし た 。 ピクシー・ダスト と は 、 原子 三 個 分 相当 の

In [ ]:
from multiprocessing import Pool
import subprocess

exe_pool = Pool(16)
pr_mor = {}
for ret in exe_pool.imap(run_mecab, pr.items()):
    pr_mor[ret[0]] = ret[1]
#print(pr_mor)
# {articleid: [bodysubのわかち書き]}

In [ ]:
print(len(pr_mor))
print(list(pr_mor.values())[:10])

In [25]:
len(pr_mor)
count = 0
for articleid, mor in pr_mor.items():
    #if count > 10000:
    #    break
    #count += 1
    print(articleid + '\t' + mor)

NIKPRLRSP038042_07012003	発表 日 : 2003 年 1 月 7 日 マイクロ ドライブ を 4 ギガバイト に 拡張 株式会社 日立 グローバルストレージテクノロジーズ ( 取締役 社長 : 宮崎 哲男 )(* 1 ) は 、 世界 最小 の ハードディスク 装置 「 マイクロ ドライブ 」 の 記憶 容量 を 4 ギガバイト に 拡張 し 、 2003 年 秋 に 発売 し ます 。 (* 1 ) 株式会社 日立 グローバルストレージテクノロジーズ は H i t a c h i GlobalStorageTechnologies ( CEO : 成瀬 淳 、 本社 機能 : 米国 カリフォルニア 州 ) の 日本 法人 として 、 2003 年 1 月 1 日 に 設立 さ れ まし た 。 同社 は ハードディスク 装置 の 開発 ・ 製造 ならびに 国内 における 販売 を 担当 し ます 。 マイクロ ドライブ は 、 標準 的 な メモリー の 規格 で ある コンパクト ・ フラッシュ TypeII の 物理 的 条件 を 満たし て おり 、 外形 寸法 は 幅 42 . 8 mm× 奥行き 36 . 4 mm× 厚 さ 5 . 0 mm という 小型 サイズ です 。 米国 IBM 社 が 2000 年 6 月 に 記憶 容量 1 ギガバイト の 製品 を 発売 し て 以来 、 大 容量 化 を 目指し て 開発 を 進め て き た 結果 、 小型 化 技術 を 活用 する こと で 、 読み書き ヘッド が 従来 製品 の 半分 の サイズ と なる 「 FEMTO (* 2 ) ヘッド 」 を 新た に 開発 する とともに 、 ヘッド が 円 板 上 を 浮上 する 高 さ ( フライト・ハイト ) も 約 40 % 下げる こと に 成功 し まし た 。 また 、 「 ピクシー・ダスト ( PixieDust )」 技術 を 利用 し て 線 記録 密度 を 大幅 に 向上 さ せ て い ます 。 更に 、 トラック 密度 の 向上 と 併せ て 面 記録 密度 も 100 億 ビット / 平方センチメートル に 向上 さ せ まし た 。 ピクシー・ダスト と は 、 原子 三 個 分 相当 の 超 薄型

In [26]:
#PRのわかち書きデータを辞書形式で保存

with open("pr_mor", 'w') as f:
    for articleid, mor in pr_mor.items():
        f.write(articleid + '\t' + mor)
        f.write('\n')
    #f.write('{' + ', '.join( ["\'" + articleid + "\': [" + ', '.join(["\'" + x + "\'" for x in mor]) + ']' for articleid, mor in pr_mor.items()] ) + '}\n')

In [29]:
# わかち書きデータの読み込み
from ast import literal_eval

pr_mor = {}
with open("pr_mor", 'r') as f:
    for line in f:
        articleid, mor = line.strip().split('\t')
        pr_mor[articleid] = mor
    #d = f.readline()
#pr_mor = literal_eval(d)
print(pr_mor['NIKPRLRSP038060_07012003'])

発表 日 : 2003 年 1 月 7 日 中国 で 単音 ・ 和音 着信 メロディ 配信 サービス ヤマハ 『 美麗 鈴 ( メロリン )』 を 開始 ヤマハ 株式会社 ( 本社 : 静岡 県 浜松 市 中沢 町 10 - 1 、 社長 : 伊藤 修 二 ) は 、 中国 最大手 の 携帯 電話 事業 者 で ある 、 中国 移動 通信 ( C h i n a M o b i l e CommunicationsCorporation 〈 略称 : CMCC 〉 本社 : 北京 市 社長 : ZhangLigui ) の 地域 会社 で ある 広東 移動 通信 有限 責任 公司 ( G u a n g d o n g M o b i l e CommunicationsCorporation 〈 略称 : GMCC ) に対し 、 広東 省 地域 で 単音 ・ 和音 ( 4 和音 ・ 16 和音 ・ 40 和音 ) 着信 メロディ 配信 サービス 、 ヤマハ 『 美麗 鈴 ( メロリン )』 を 、 2003 年 1 月 1 日 から 提供 開始 し まし た 。 中国 で は ここ 数 年 で 携帯 電話 の 普及 が 急速 に 進ん で い ます 。 人口 約 13 億 人 に対し 、 現在 利用 者 数 は 世界 第 一 位 の 約 2 億 人 に のぼり 、 大きな 市場 性 が ある と 考え られ ます 。 今回 サービス を 開始 する 広東 省 は 、 人口 約 9000 万 人 に対し 、 携帯 電話 加入 者 数 は 約 2700 万 人 と 中国 国内 で は 最も 普及 率 の 高い 地域 です 。 加え て 商業 が 盛ん で 、 携帯 電話 端末 による データ通信 も 非常 に 多い こと から 、 今後 の 着信 メロディ 市場 の さら なる 成長 が 期待 さ れ て い ます 。 また 、 当社 は 既に 香港 で の 着信 メロディ サービス を 開始 し て い ます が 、 広東 省 は 地理 的 に も 香港 に 隣接 し 、 文化 の 影響 を 多く 受け て いる こと から 、 これ まで 培っ た 経験 ・ ノウハウ を 活用 できる と 考え て い ます 。 なお 、 今回 の サービス 展開 にあたって 

In [30]:
#わかち書き形式を形態素のリストに分割
for articleid, mor in pr_mor.items():
    pr_mor[articleid] = mor.split(' ')
    
print(pr_mor['NIKPRLRSP038060_07012003'])

['発表', '日', ':', '2003', '年', '1', '月', '7', '日', '中国', 'で', '単音', '・', '和音', '着信', 'メロディ', '配信', 'サービス', 'ヤマハ', '『', '美麗', '鈴', '(', 'メロリン', ')』', 'を', '開始', 'ヤマハ', '株式会社', '(', '本社', ':', '静岡', '県', '浜松', '市', '中沢', '町', '10', '-', '1', '、', '社長', ':', '伊藤', '修', '二', ')', 'は', '、', '中国', '最大手', 'の', '携帯', '電話', '事業', '者', 'で', 'ある', '、', '中国', '移動', '通信', '(', 'C', 'h', 'i', 'n', 'a', 'M', 'o', 'b', 'i', 'l', 'e', 'CommunicationsCorporation', '〈', '略称', ':', 'CMCC', '〉', '本社', ':', '北京', '市', '社長', ':', 'ZhangLigui', ')', 'の', '地域', '会社', 'で', 'ある', '広東', '移動', '通信', '有限', '責任', '公司', '(', 'G', 'u', 'a', 'n', 'g', 'd', 'o', 'n', 'g', 'M', 'o', 'b', 'i', 'l', 'e', 'CommunicationsCorporation', '〈', '略称', ':', 'GMCC', ')', 'に対し', '、', '広東', '省', '地域', 'で', '単音', '・', '和音', '(', '4', '和音', '・', '16', '和音', '・', '40', '和音', ')', '着信', 'メロディ', '配信', 'サービス', '、', 'ヤマハ', '『', '美麗', '鈴', '(', 'メロリン', ')』', 'を', '、', '2003', '年', '1', '月', '1', '日', 'から', '提供', '開始', 'し', 'まし', 'た', '。', '中国'

In [268]:
from gensim import models
from gensim.models.doc2vec import LabeledSentence

In [8]:
len(pr_mor)

10

In [284]:
def generate_sentences(pr_mor):
    for articleid, mor in pr_mor.items():
        yield LabeledSentence(words=mor, tags=[articleid])

In [304]:
sentences = generate_sentences(pr_mor)
model = models.Doc2Vec(sentences, dm=1, size=300, window=15, alpha=.025, min_alpha=.025, min_count=1, sample=1e-6)

# dm = 1で高精度の学習
# sizeは文書のベクトル表現の次元

epoch_num = 100
for epoch in range(epoch_num):
    print('Epoch: {}'.format(epoch + 1))
    model.train(sentences)
    model.alpha -= (0.025 - 0.0001) / (epoch_num - 1)
    model.min_alpha = model.alpha

Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
Epoch: 30
Epoch: 31
Epoch: 32
Epoch: 33
Epoch: 34
Epoch: 35
Epoch: 36
Epoch: 37
Epoch: 38
Epoch: 39
Epoch: 40
Epoch: 41
Epoch: 42
Epoch: 43
Epoch: 44
Epoch: 45
Epoch: 46
Epoch: 47
Epoch: 48
Epoch: 49
Epoch: 50
Epoch: 51
Epoch: 52
Epoch: 53
Epoch: 54
Epoch: 55
Epoch: 56
Epoch: 57
Epoch: 58
Epoch: 59
Epoch: 60
Epoch: 61
Epoch: 62
Epoch: 63
Epoch: 64
Epoch: 65
Epoch: 66
Epoch: 67
Epoch: 68
Epoch: 69
Epoch: 70
Epoch: 71
Epoch: 72
Epoch: 73
Epoch: 74
Epoch: 75
Epoch: 76
Epoch: 77
Epoch: 78
Epoch: 79
Epoch: 80
Epoch: 81
Epoch: 82
Epoch: 83
Epoch: 84
Epoch: 85
Epoch: 86
Epoch: 87
Epoch: 88
Epoch: 89
Epoch: 90
Epoch: 91
Epoch: 92
Epoch: 93
Epoch: 94
Epoch: 95
Epoch: 96
Epoch: 97
Epoch: 98
Epoch: 99
Epoch: 100


In [297]:
#モデルのセーブ
model.save('doc2vec.model')

In [287]:
#モデルのロード
#model = models.Doc2Vec.load('doc2vec.model')

In [301]:
#最も近い文章
count = 0
for articleid, bodysub in pr.items():
    if count >= 1000:
        break
    count += 1 
    print(articleid, model.docvecs.most_similar(articleid, topn=1))

NIKPRLRSP038042_07012003 [('NIKPRLRSP040922_19022003', 0.21206073462963104)]
NIKPRLRSP038060_07012003 [('NIKPRLRSP050761_15072003', 0.2186652272939682)]
NIKPRLRSP038177_08012003 [('NIKPRLRSP058664_06112003', 0.22826343774795532)]
NIKPRLRSP038195_09012003 [('NIKPRLRSP048108_09062003', 0.22477564215660095)]
NIKPRLRSP038225_09012003 [('NIKPRLRSP046049_09052003', 0.2223157286643982)]
NIKPRLRSP038228_09012003 [('NIKPRLRSP045787_07052003', 0.21367472410202026)]
NIKPRLRSP038237_09012003 [('NIKPRLRSP052992_19082003', 0.21944114565849304)]
NIKPRLRSP038242_09012003 [('NIKPRLRSP050523_10072003', 0.22044670581817627)]
NIKPRLRSP038250_09012003 [('NIKPRLRSP050948_16072003', 0.23530933260917664)]
NIKPRLRSP038265_09012003 [('NIKPRLRSP059590_19112003', 0.25983431935310364)]
NIKPRLRSP038296_10012003 [('NIKPRLRSP055523_24092003', 0.21352384984493256)]
NIKPRLRSP038353_10012003 [('NIKPRLRSP059565_19112003', 0.2212894856929779)]
NIKPRLRSP038443_14012003 [('NIKPRLRSP043523_28032003', 0.2425188273191452)]
NIK

-> だいたいどれも最大で0.25くらいの値。sparseであることを表しているか。

In [302]:
#0.33の例
pr["NIKPRLRSP055443_24092003"]

'発表日:2003年9月22日-つゆに丸大豆醤油、おあげが新しくなって、きつねそばリニューアル-どんぶり型和風カップめん「日清のどん兵衛 きつねそば」リニューアル新発売のご案内 日清食品株式会社(社長:安藤宏基)は、「日清のどん兵衛 きつねそば」を2003年10月6日(月)より、リニューアル新発売いたします。■ リニューアルの意図 弊社では、和風カップめんの最需要期である年末に向け「どん兵衛」ブランドのラインナップの強化、拡充を図ることを目的に、「どん兵衛きつねそば」のつゆの醤油を丸大豆醤油に変更、あげの食感の改良をするなど「品質の強化」をいたします。これにより、どん兵衛ブランド全体の活性化を図ります。■ リニューアルのポイント1.つゆの品質強化 つゆに使用している醤油を丸大豆醤油に変更いたします。素材のおいしさまるごとの丸大豆だから、つゆに素材の旨さがいきてます。2.あげの強化 ジューシィな丸大豆おあげが、外側は歯ごたえのあるもっちりした食感、内側はふっくら炊き込んだ食感になりました。■ 商品特徴(1)め ん そばらしい彩りとしなやかで、つるみがある喉ごしが自慢のシャキシャキそばです。(2)つ ゆ 素材にこだわり、焼津産かつお節と北海道産昆布をふんだんに使った上質なダシに、まるみのある味わいの丸大豆醤油を使用、あげからの甘みが渾然一体となったおいしさのつゆです。(3)具 材 ダシでふっくら炊き上げたまろやかな味、表面もっちり、中はふっくらな本格派のおあげは丸大豆100%、遺伝子組み換えでない大豆を使用しています。アクセントとしてナルト、ねぎが彩りを添えます。■ 商品概要 * 関連資料 参照'

In [303]:
pr["NIKPRLRSP044826_17042003"]

'発表日:2003年4月17日セガ社への合併提案に関する報道について 報道されたように、弊社からセガ社に対し正式に合併を申し入れていることは事実です。 弊社とセガ社とは2001年9月20日より、アミューズメント事業分野における包括業務提携を結んでおります。その後、昨年より全体統合に向けての話を進めておりましたが、セガ社の希望により一時中断しておりました。 今回改めて、長年ライバルとして競争しあいアミューズメント業界を牽引してきたセガ社と弊社が、全ての事業分野において力を合わせていくことが最大の統合効果を発揮し、アミューズメント業界の発展にも寄与するものと確信し、合併についての申し入れをいたしました。以 上'

-> ？？？

In [319]:
print(model)

Doc2Vec(dm/m,d300,n5,w15,s1e-06,t3)


In [328]:
model.docvecs["NIKPRLRSP055443_24092003"]

array([ -1.58279680e-03,   2.38512686e-04,  -1.23063545e-03,
         1.39107564e-04,  -1.27170270e-03,   1.15015951e-03,
         7.00502656e-04,   1.48398289e-03,   3.95193667e-04,
         8.54048179e-04,  -9.48425848e-04,   1.02541759e-04,
        -1.53797446e-03,   6.06486923e-04,   4.88212536e-04,
        -1.26947172e-03,   1.48671540e-03,   5.18104527e-04,
        -1.92121370e-04,   1.12238387e-03,   1.08449766e-03,
        -1.30022920e-04,  -1.40532968e-03,   1.63994858e-03,
        -2.46805197e-04,  -1.46405320e-04,  -4.34182322e-04,
        -7.00066972e-04,  -6.66309570e-05,   1.56586932e-03,
         9.76298121e-04,   7.17577175e-04,  -8.59859749e-04,
         1.34676462e-03,   2.65779265e-04,  -4.83891519e-04,
         1.30555104e-03,  -1.23268156e-03,  -1.26635353e-03,
         9.94446571e-04,  -1.47771009e-03,  -1.32763351e-03,
         1.00087700e-03,   1.00228016e-03,   5.88184281e-04,
        -8.32919904e-04,   9.34869808e-04,  -9.47714667e-04,
         8.51489953e-04,

-> model.docvecs を学習器の入力とすれば良いか

### クラスターマシンでの計算結果例

ローカルだとメモリが足りなかったので結果だけ記載

1行目はパラメタ(dm, size, window, alpha, min_alpha, min_count, sample, workers, epoch_num)

2行目以降はマッチしたPRのペアとそのsimilarity

similarity ~ 1の数例と、similarity <= 0.36 の大部分の2群に分割されてしまっている -> 学習がうまくいっていない？

similarity ~ 1のものも、PR本文を見ると謎な結果になっている

NIKPRLRSP346294_27092013	20130927	発表日：2013年9月27日Dell　Graphic　Proシリーズに『Inspiron　１５　７０００シリーズ』と『Inspiron　１７　７０００シリーズ』の２機種が新登場～薄型設計ボディに最新テクノロジーとAdobe　Photoshop　Elements　１１　＆　Premiere　Elements　１１を搭載。届いたその日から最高のクリエイティブ体験を～　Dellは、動画像編集に強いDell　Graphic　ProシリーズにInspironノートブック新製品『Inspiron　１５　７０００シリーズ』と『Inspiron　１７　７０００シリーズ』を加え、本日９月２７日（金）より販売開始いたします。　「Dell　Graphic　Proシリーズ」とは、多くの制作現場やプロのクリエイターたちから支持を受けているDellが、個人のお客様のより快適で充実したデジタルクリエイティブライフを実現するために、映像スタジオ仕様並みのスペックを備える製品を厳選しシリーズ化したものです。「Adobe　Photoshop　Elements　１１　＆　Premiere　Elements　１１」を標準搭載しており、お手元に届いたその日から動画像編集をお楽しみ頂けます。　※参考画像は、添付の関連資料を参照　これまで「Dell　Graphic　Proシリーズ」ノートブックとして『Inspiron　１５　Special　Edition』と『Inspiron　１７　Special　Edition』をラインアップしていましたが、より薄型の設計ながら最新テクノロジー搭載した新製品『Inspiron　１５／１７　７０００シリーズ』の２機種を追加し、「Dell　Graphic　Proシリーズ」はさらなる進化を遂げています。　『Inspiron　１５／１７　７０００シリーズ』は共に、スタイリッシュかつ高級感のある薄型アルミニウムボディに、最新のCPUとグラフィックカードを搭載し、パワフルなパフォーマンス性能を発揮すると共に、Windows　８に最適な１０点マルチタッチ対応のディスプレイによる快適な操作性を実現しています。※製品画像は、添付の関連資料「製品画像（１）」を参照＜『Inspiron　１５　７０００シリーズ』の主な特長＞　自由に持ち運びが可能なハイスペック１５インチノート。室内外で最高のクリエイティブ体験を提供　・スタイリッシュかつ高級感のある削り出しアルミニウムボディ　・Adobe　Photoshop　Elements　１１　＆　Premiere　Elements　１１を標準搭載　・最新の第４世代インテル　Core　i５／i７プロセッサーを搭載　・最新のグラフィックスカード　NVIDIA(R)　GeForce(R)　GT　７５０M　２GB　DDR５を標準搭載　・最大８GB（４GBx２）　デュアルチャネル　DDR３L　メモリ（１６００MHz）　・１５．６インチ　フルHDマルチタッチディスプレイ（１９２０×１０８０）搭載モデルも選択可能　・最大１TB　HDD（５４００回転）　・豊富な各種ポート：HDMI　v１．４　（１）、USB　３．０　（３）、USB　３．０　PowerShare　（１）、RJ４５　Ethernet　（１）、マルチメディアカードリーダー　（１）、マイク／ヘッドフォン兼用（１）　・ワイヤレスにはDell　Wireless　８０２．１１a／b／g／n、　Bluetooth　v４．０搭載　・サイズ：高さ２２．１５mm／幅３７９．４mm／奥行き２５４．８mm　・重量：２．６kg　※製品画像は、添付の関連資料「製品画像（２）」を参照＜『Inspiron　１７　７０００シリーズ』の主な特長＞　１７インチ大画面液晶で高画質な映像の編集も自由自在に　・スタイリッシュかつ高級感のある削り出しアルミニウムボディ　・Adobe　Photoshop　Elements　１１　＆　Premiere　Elements　１１を標準搭載　・最新の第４世代インテル　Core　i５／i７プロセッサーを搭載　・最新のグラフィックスカード　NVIDIA(R)　GeForce(R)　GT　７５０M　２GB　DDR５を標準搭載　・最大１６GB（８GBx２）　２　DIMM　DDR３L　メモリ（１６００Mhz）　・１７．３インチ　フルHDマルチタッチディスプレイ（１９２０x１０８０）搭載モデルも選択可能　・１TB　HDD（５４００回転）　・豊富な各種ポート：HDMI　v１．４　（１）、USB　３．０　（３）、USB　３．０　PowerShare　（１）、RJ４５　Ethernet　（１）、マルチメディアカードリーダー　（１）、マイク／ヘッドフォン兼用（１）　・ワイヤレスにはDell　Wireless　８０２．１１a／b／g／n、　Bluetooth　v４．０搭載　・サイズ：高さ２７．７mm／幅４１２．４mm／奥行き２６９．４mm　・重量：３．２９kg　※製品画像は、添付の関連資料「製品画像（３）」を参照　販売開始日：２０１３年９月２７日（金）　製品ページ：Inspiron　１５　７０００シリーズ　http://www.dell.com/jp/p/inspiron-15-7537/pd.aspx　Inspiron　１７　７０００シリーズ　http://www.dell.com/jp/p/inspiron-17-7737/pd.aspx＜モデル構成一覧＞　※添付の関連資料を参照　■DELL　ロゴは、米国　Dell　Inc．　の商標または登録商標です。　■その他の社名および製品名は、各社の商標または登録商標です。　■記載内容は、２０１３年９月２７日時点のものです。

と

NIKPRLRSP251168_13052010	20100513	発表日：2010年5月13日２０１１年以降も今あるアナログカーテレビで地デジ（ワンセグ）が見られる！アナログカーテレビ用ワンセグセットを新発売！ＡＶ接続不要で簡単に設置できます　マスプロ電工は，アナログカーテレビでワンセグを視聴できるアナログカーテレビ用ワンセグセット２機種を５月下旬から発売します。　今回発売するワンセグセットは，２０１１年以降，使用できなくなるアナログカーテレビを救済する製品として，複雑なＡＶ接続が不要で，簡単にワンセグが見られるよう開発されたものです。本セットは，アンテナでワンセグを受信（１）（※　関連資料参照）し，ワンセグチューナーを内蔵した本体でワンセグ（デジタル信号）をアナログ信号に変換（２）して再送信（３）しますから，既設のアナログカーテレビでワンセグ映像が見られます。　マスプロ電工は，このワンセグセットを市場投入することで，住宅用テレビだけでなく，カーテレビの地デジ化もサポートします。■開発の背景と意図　２０１１年７月２４日の地デジ完全移行後，アナログチューナー搭載カーナビゲーションシステム（以下カーナビ）やアナログカーテレビなどは，テレビ放送が視聴できなくなります。そのため，カーナビやアナログカーテレビなどは，約１年余りで地デジ化をする必要があります。しかし，市販の車載用ワンセグチューナーはＡＶ接続する製品が主流で，カーナビやアナログカーテレビに接続する際，コンソールパネルの取外しが必要になるなど作業が難しいため，地デジ化が進んでいません。　そこで，コンソールパネルを外すといった手間な作業が不要で，簡単にアナログ　　チューナー搭載カーナビやアナログカーテレビでワンセグが見られるアナログカー　テレビ用ワンセグセットを開発しました。■主な特長●簡単取付　ワンセグをアンテナで受信し，本体でＶＨＦのアナログ信号に変換して，既設のアンテナに再送信（特許出願中）しますから，アナログチューナー搭載カーナビやアナログカーテレビで簡単にワンセグを見られます。　ＡＶコード等での接続がないためコンソールパネルを外しての作業が不要になり，簡単に設置できます。※　　※　車外に取付けられているロッド式のアンテナやＧＰＳ一体型のアンテナには使用できません。●自動スキャン機能　遠出をしたときに放送エリアを外れても，自動的にその地域の放送チャンネルを探す機能（自動スキャン機能）を装備していますから，ワンセグを受信できる地域なら，いつでも，どこでも視聴できます。●ホームチャンネル登録機能　普段視聴している地域のチャンネルを「ホームチャンネル」として登録できますから，遠出をしたとき，チャンネルを変更しても，リモコンの「ホーム」ボタンを押すだけで，簡単に普段視聴しているチャンネルに戻すことができます。●簡易番組表　画面に簡易番組表を表示できますから，番組を確認することができます。●ズーム（画面拡大）機能　標準テレビ（４：３）でワンセグを視聴した場合，番組によってはテレビ画面より小さな映像（額縁のように映像の周りが黒く縁取られた状態）になることがあります。　このような額縁画像放送でも，ズーム（画面拡大）機能によりフル画面表示できます。●環境に配慮した製品。だからｅｃｏ。　無鉛はんだの採用，カドミウム・水銀などの不使用により，ＥＵ（欧州連合）の特定有害物質使用規制（ＲｏＨＳ指令）に対応した，環境負荷の少ない製品です。　※　詳細は、関連資料参照

---
---

NIKPRLRSP108550_24082005	20050824	発表日：２００５年８月２４日高千穂交易、ＡＴＭ向け後方確認用ミラーを大手都市銀行に納入～「安全・安心・快適」な社会づくりに貢献～　高千穂交易株式会社（本社：東京都新宿区、代表取締役社長：山村秀彦）は、薄型で凸面鏡と同等の広い範囲が見える「ＡＴＭ後方確認用ミラー」が大手都市銀行のＡＴＭ（現金自動預け払い機）に導入されたことを発表します。　昨今、多くの金融機関が情報漏えい防止や本人確認のために、キャッシュカードのＩＣ化や、生体認証（静脈や虹彩を利用した認証）システムの導入を進めていますが、暗証番号の覗き見や偽造キャッシュカードによる犯罪は依然として多発しています。また、ＡＴＭでの入出金時に利用者が抱く「背後から覗かれている気がする」等の不安感解消が課題とされていました。　高千穂交易では、ＡＴＭのセキュリティ強化、利用者の快適さ・利便性確保の対策として「後方確認用ミラー」を各金融機関に提案してまいりました。金融機関が薄型のミラーをＡＴＭ前面に貼るだけで、利用者は背後に人がいることが分かり、安心してＡＴＭを利用することができます。（取付例写真ご参照）　「後方確認用ミラー」の取扱いを開始した２００４年１０月から２００５年７月までの累計で約３６，０００枚（約７７百万円）がメガバンクのＡＴＭに導入済みです。　高千穂交易は、今後、国内のＡＴＭ約１５万台のうち、後方確認用ミラー又は一般のミラーが未採用のＡＴＭ約７～８万台の需要を見込んでいます。併せて、ＡＴＭメーカーへ新規開発ＡＴＭに「後方確認用ミラー」の標準採用促進を働きかけ、年間２万台分の販売を計画しています。　また、券売機、無人契約機、証明書発行機等の、暗証番号や現金を扱うＡＴＭ　以外の自動機アプリケーションへの展開も図る予定です。【ＡＴＭ後方確認用ミラーの特長】　・フラット、小型、薄型の形状・・・ＡＴＭの仕様を変更せず僅かなスペースに設置が可能　・広い視野・・・平面鏡と同様フラットで、凸面鏡と同等の広い視野　・設置が容易（工事不要）　・不審者へプレッシャー、犯罪抑止効果【ＡＴＭ後方確認用ミラー】　（＊　参考画像参照）

と

NIKPRLRSP329519_01022013	20130201	発表日：2013年2月1日～学生×障がい福祉施設、連携企画～廃棄予定の衣料を再利用した、世界にひとつだけの色合わせが楽しめるシュシュ販売開始　株式会社フェリシモは、廃棄処分予定の衣料品を教育機関に提供し、新たな価値を持つアイテムへと再生させる「リボーンス　ドリームボックスプロジェクト」を２０１０年に開始しました。　このプロジェクトを通じて、２０１１年８月から、ファッションデザインの専門学校生たちが企画した商品を、京都市の障がい福祉施設が生産し、京都市のアンテナショップとフェリシモの通販サイトで販売する取り組みがスタートしました。　第１弾のブックカバーに続き、第２弾として、さまざまな生地を組み合わせて作ったシュシュの販売を２月１日（金）よりクラソポットにて開始いたします。　シュシュのアイデアは、マロニエファッションデザイン専門学校（大阪市北区）の学生たちによるもの。生産は、京都市の“はあと・フレンズ・ストア”にコーディネイトいただいた３ヵ所の障がい福祉施設にて行います。▼商品ラインナップ　トップスやスカートなどの衣料品から生地を切り取り、柄を組み合わせました。同じものは二つとない、世界にひとつだけの色合わせが楽しめるシュシュです。衣服が、以前とは違う形で、人々を着飾る商品に生まれ変わりました。　人・もの・社会をつなぐ　ひとつひとつがお気に入り！　やさしい気持ちの手仕事が伝わるシュシュ　製作：工房ソルト（http://www.kyoto-hotheart.jp/modules/contents/?content_id=130）　人・もの・社会をつなぐ　ひとつひとつがお気に入り！　笑顔が満タンのシュシュ　製作：ピースフルリンク（http://www.kyoto-hotheart.jp/modules/contents/?content_id=130）　人・もの・社会をつなぐ　ひとつひとつがお気に入り！　陽だまりのシュシュ　製作：山科工房（http://yamashinakoubou.blogzine.jp/）　＊商品画像は添付の関連資料を参照▼価格　５９０円（税込み）　※デザインの異なる２個セット▼販売　※数量限定で販売　【ウェブサイト】フェリシモ　クラソポット　　　　　　　　　　http://www.felissimo.co.jp/kraso/pot/　【店舗】はあと・フレンズ・ストア　　　　　　http://www.hatarakimahyo.jp/hfp/　　　　　　（京都市下京区四条通室町東入函谷鉾町８０番地京都産業会館　地階）▼発売日　２０１３年２月１日（金）▼企画～販売までの過程　フェリシモの廃棄処分予定の衣料品を材料に、マロニエファッションデザイン専門学校の学生たちが、障がいのある人が生産できる商品アイテムを考え、デザインの仕様書と試作品も制作。フェリシモプロジェクトスタッフが商品選定を行い、商品化アイテムを決定しました。一方で、京都市が設置する障害のある人が広く関わった製品のアンテナショップ「はあと・フレンズ・ストア」が商品生産の施設を公募。審査の結果、３つの施設が今回のシュシュの生産を担当することとなりました。販売はフェリシモのウェブサイトと、「はあと・フレンズ・ストア」にて同時にスタートします。■リボーンス　ドリームボックスプロジェクト　http://www.felissimo.co.jp/kraso/act/rebornth/?iid=p_kr_111216_ACTRE　フェリシモの販売基準に適さなかった商品や、季節の経過などで販売しにくくなった商品を、処分するのではなく、箱単位で教材として学校法人など団体に提供し、学生やクリエイターの力によって新たな価値を持つアイテムへと再生（リボーン）させる取り組み。２０１３年１月現在、１４の教育機関に参加いただいています。このプロジェクトにより、これまでに累計３０００枚以上の衣料品が廃棄処分とならず、有効に活用されました。